In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

data= pd.read_csv('training_cleaned.csv')


data.columns=['text','sentiment']
data=data.astype(str)


Using TensorFlow backend.


In [2]:
#parameters.  max number of features after vectorization ==2000. essentially no limit
max_features = 2000
#tokenize 
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
#embed_dim= dimension of embedding. Words represented as a vector of numbers of 'embed_dim'
embed_dim = 64
#lstm hidden layers
lstm_out = 196


In [3]:
#rnn model
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 64)            128000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 19, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               204624    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 333,018
Trainable params: 333,018
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
#splitting train and test.  2/3 train 1/3 test
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(21398, 19) (21398, 2)
(10540, 19) (10540, 2)


In [5]:

#batch size=number of training smaples considered before updating weights
batch_size = 32
#fit model
#epochs= number of iterations.
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2)

#accuracy is rounded to the nearest percent
score,acc = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))



Epoch 1/3
 - 28s - loss: 0.5628 - acc: 0.7065
Epoch 2/3
 - 27s - loss: 0.4903 - acc: 0.7673
Epoch 3/3
 - 27s - loss: 0.4717 - acc: 0.7790
score: 0.50
acc: 0.75


In [14]:
def classify(row):
    if row[0] == row[1]:
        val = 0
    elif row[0] > row[1]:
        val = 0
    else:
        val = 1
    return val

In [11]:
df= pd.DataFrame(preds)

In [15]:
df['Predicted'] = df.apply(classify,axis=1)

In [23]:
y_test = pd.DataFrame(Y_test)
y_test["Actual"] = y_test.apply(classify,axis=1)

In [26]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
df_dl =pd.DataFrame()
preds = df['Predicted']
y_test = y_test["Actual"]
accuracy = accuracy_score(y_test, preds)
model_precision, model_recall, model_fscore, model_support = precision_recall_fscore_support(y_test,preds)
#bestngram = clf.best_params_['vectorizer__ngram_range']
#bestnfeatures = clf.best_params_['vectorizer__max_features']
df_loop_stack = pd.DataFrame(['RNN',accuracy,model_precision[0],model_recall[0],model_fscore[0]])
df_dl = df_dl.append(df_loop_stack.T, ignore_index = True)
df_dl.columns = ['Model','Accuracy','Precision','Recall','Fscore']
print("Validation Data Model")
df_dl

Validation Data Model


,Model,Accuracy,Precision,Recall,Fscore
0,RNN,0.754175,0.762081,0.740579,0.751176
